In [ ]:
# Import libraries
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# Specify path to training, validation, and testing data
train_path = 'path to your training dataset directory'
val_path = 'path to your validation dataset directory'
test_path = 'path to your testing dataset directory'

In [ ]:
# Normalize images (you can also augment training data if needed here)
train_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

target_size = (256,256)

# Prepare batches of data for training
train_data_gen = train_datagen.flow_from_directory(train_path,
                                               target_size=target_size,
                                               batch_size=40,
                                               class_mode='binary')
# Prepare batches of data for validation
val_data_gen = val_datagen.flow_from_directory(val_path,
                                               target_size=target_size,
                                               batch_size=20,
                                               class_mode='binary',
                                               shuffle=False)
# Prepare batches of data for testing on unknown samples (not used for training/validation)
test_data_gen = test_datagen.flow_from_directory(test_path,
                                               target_size=target_size,
                                               batch_size=20,
                                               class_mode='binary',
                                               shuffle=False)

In [ ]:
# Visualize sample of your training images (optional)
# The next function returns a batch from the dataset, use image data, discard the labels
sample_training_images, _ = next(train_data_gen)

# This function will plot images in the form of a grid with 4 rows and 4 columns 
def plotImages(images_arr):
    fig, axes = plt.subplots(4, 4, figsize=(10,10))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Plot some images from training data    
plotImages(sample_training_images[:16])

In [ ]:
# Build your CNN model

input_shape = (256,256,3)

model = Sequential()

# Add convolutional/pooling layers

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=input_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=input_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),input_shape=input_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add flatten layer
model.add(Flatten())

# Add dense/dropout layers + activation functions

model.add(Dense(128))
model.add(Activation('relu'))# relu activation function for deep layer
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('sigmoid'))# sigmoid activation function for last layer for binary classification

# Print summary of your model
model.summary() 

In [ ]:
# Compile and train your model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
history = model.fit_generator(train_data_gen,
                              steps_per_epoch=200,
                              epochs=epochs,
                              validation_data=val_data_gen,
                              validation_steps=50)

In [ ]:
# Plot model accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Test your model on unknown samples (testing data)
test_loss, test_acc = model.evaluate_generator(test_data_gen, steps=50)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)